In [1]:
import os
import itertools
import numpy as np
import pandas as pd
from PIL import Image

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import roc_auc_score

import timm

CLASSES = [
    "No Finding", "Atelectasis", "Cardiomegaly", "Effusion", "Infiltration",
    "Mass", "Nodule", "Pneumonia", "Pneumothorax", "Consolidation",
    "Edema", "Emphysema", "Fibrosis", "Pleural_Thickening", "Hernia"
]

class GammaCorrectionTransform:
    def __init__(self, gamma=1.0):
        self.gamma = gamma

    def __call__(self, img):
        img_np = np.array(img).astype(np.float32) / 255.0
        corrected = np.power(img_np, self.gamma)
        corrected = np.clip(corrected * 255, 0, 255).astype(np.uint8)
        return Image.fromarray(corrected)

df = pd.read_csv("/student/csc490_project/shared/labels.csv")
df["label_list"] = df["Finding Labels"].apply(lambda x: x.split("|"))

mlb = MultiLabelBinarizer(classes=CLASSES)
labels_array = mlb.fit_transform(df["label_list"])
df["labels"] = list(labels_array)

# Patient-level split: 70% train, 10% val, 20% test
unique_patients = df["Patient ID"].unique()
np.random.seed(42)
np.random.shuffle(unique_patients)

train_end = int(0.7 * len(unique_patients))
val_end = int(0.8 * len(unique_patients))

train_patients = unique_patients[:train_end]
val_patients = unique_patients[train_end:val_end]
test_patients = unique_patients[val_end:]

train_df = df[df["Patient ID"].isin(train_patients)].reset_index(drop=True)
val_df = df[df["Patient ID"].isin(val_patients)].reset_index(drop=True)
test_df = df[df["Patient ID"].isin(test_patients)].reset_index(drop=True)

class ChestXrayDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.df.iloc[idx]["Image Index"])
        image = Image.open(img_path).convert("L")
        labels = torch.tensor(self.df.iloc[idx]["labels"], dtype=torch.float)
        if self.transform:
            image = self.transform(image)
        return image, labels

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

img_dir = "/student/csc490_project/shared/preprocessed_images/preprocessed_images"
train_dataset = ChestXrayDataset(train_df, img_dir, train_transform)
val_dataset = ChestXrayDataset(val_df, img_dir, val_transform)
test_dataset = ChestXrayDataset(test_df, img_dir, val_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

models = {
    'maxvit': timm.create_model('maxvit_rmlp_base_rw_224.sw_in12k_ft_in1k', pretrained=False, num_classes=15),
    'densenet': timm.create_model('densenet121', pretrained=False, num_classes=15),
    'coatnet': timm.create_model('coatnet_2_rw_224.sw_in12k_ft_in1k', pretrained=False, num_classes=15),
    'vgg19': timm.create_model('vgg19.tv_in1k', pretrained=False, num_classes=15),
    'swin': timm.create_model('swin_large_patch4_window7_224', pretrained=False, num_classes=15),
    'convnext': timm.create_model('convnext_large.fb_in22k', pretrained=False, num_classes=15)
}

models['maxvit'].load_state_dict(torch.load('/student/csc490_project/shared/new_split_models/gamma_maxvit_rmlp_base_rw_224.sw_in12k_ft_in1k_model.pth'))
models['densenet'].load_state_dict(torch.load('/student/csc490_project/shared/new_split_models/gamma_densenet121_model.pth'))
models['coatnet'].load_state_dict(torch.load('/student/csc490_project/shared/new_split_models/gamma_coatnet_2_rw_224.sw_in12k_ft_in1k_model.pth'))
models['vgg19'].load_state_dict(torch.load('/student/csc490_project/shared/new_split_models/gamma_vgg19.tv_in1k_model.pth'))
models['swin'].load_state_dict(torch.load('/student/csc490_project/shared/new_split_models/gamma_swin_large_patch4_window7_224_model.pth'))
models['convnext'].load_state_dict(torch.load('/student/csc490_project/shared/new_split_models/gamma_convnext_large.fb_in22k_model.pth'))

for model in models.values():
    model.to(device)
    model.eval()

/tmp/ipykernel_574928/3906317376.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  models['maxvit'].load_state_dict(torch.load('/student/csc490_project/shared/new_split_

In [2]:
# Function to evaluate a model on the test set
def evaluate_model(model, dataloader, device):
    all_targets = []
    all_outputs = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            outputs = torch.sigmoid(outputs)

            all_targets.append(labels.cpu().numpy())
            all_outputs.append(outputs.cpu().numpy())

    all_targets = np.vstack(all_targets)
    all_outputs = np.vstack(all_outputs)

    auroc_scores = []
    for i in range(len(CLASSES)):
        try:
            score = roc_auc_score(all_targets[:, i], all_outputs[:, i])
            auroc_scores.append(score)
        except ValueError:
            auroc_scores.append(float('nan'))

    return auroc_scores

# Loop through each model and print AUROC scores
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    auroc_scores = evaluate_model(model, test_loader, device)

    print(f"AUROC Scores for {model_name}:")
    for cls, score in zip(CLASSES, auroc_scores):
        print(f"{cls}: {score:.4f}")
    print(f"Mean AUROC: {np.nanmean(auroc_scores):.4f}")
    print("-" * 50)


Evaluating maxvit...
AUROC Scores for maxvit:
No Finding: 0.7731
Atelectasis: 0.8111
Cardiomegaly: 0.9046
Effusion: 0.8810
Infiltration: 0.7120
Mass: 0.8526
Nodule: 0.7728
Pneumonia: 0.7459
Pneumothorax: 0.8629
Consolidation: 0.8061
Edema: 0.9058
Emphysema: 0.9132
Fibrosis: 0.8269
Pleural_Thickening: 0.7841
Hernia: 0.8850
Mean AUROC: 0.8291
--------------------------------------------------
Evaluating densenet...
AUROC Scores for densenet:
No Finding: 0.7862
Atelectasis: 0.8191
Cardiomegaly: 0.8957
Effusion: 0.8837
Infiltration: 0.7149
Mass: 0.8529
Nodule: 0.7654
Pneumonia: 0.7757
Pneumothorax: 0.8694
Consolidation: 0.8031
Edema: 0.9054
Emphysema: 0.9212
Fibrosis: 0.8032
Pleural_Thickening: 0.8102
Hernia: 0.8636
Mean AUROC: 0.8313
--------------------------------------------------
Evaluating coatnet...
AUROC Scores for coatnet:
No Finding: 0.7845
Atelectasis: 0.8123
Cardiomegaly: 0.8708
Effusion: 0.8831
Infiltration: 0.7083
Mass: 0.8406
Nodule: 0.7661
Pneumonia: 0.7619
Pneumothorax: 0.